In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xtrack as xt
import xpart as xp
import xcoll as xc
import xobjects as xo

In [2]:
def install_tidp(line, block_mvt=29e-3):
    tidp_ap_tot = 147e-3
    line.discard_tracker()
    tidp = xc.EverestCollimator(length=4.3, material=xc.materials.Carbon, jaw_L= tidp_ap_tot/2 + block_mvt, jaw_R = -tidp_ap_tot/2 + block_mvt)
    line.collimators.install(names=['tidp.11434'], elements=[tidp])
    return tidp

In [13]:
context = xo.ContextCpu()
line = xt.Line.from_json('../../injection_lines/sps_with_aperture_inj_q20_beam_sagitta5.json')

line.vars['qph_setvalue'] = 0.5
line.vars['qpv_setvalue'] = 0.5
line.vars['qh_setvalue'] = line.vars['qx0']._value + 0.05
line.vars['qv_setvalue'] = line.vars['qy0']._value + 0.05


cavity_elements, cavity_names = line.get_elements_of_type(xt.Cavity)

for name in cavity_names:
    line[name].frequency = 200e6
    line[name].lag = 180
line['acl.31735'].voltage = 0 #setting 800 cav to 0V
line['actcse.31632'].voltage = 3.0e6

tt = line.get_table()
tw = line.twiss()

Loading line from dict:   0%|          | 0/36404 [00:00<?, ?it/s]

Done loading line from dict.           


In [14]:
# context = xo.ContextCpu()

aperture_mvt = 0
i = 100
num_particles  = 1000
plane = 'DPneg'
sweep = 6000
sweep = -abs(sweep) if plane == 'DPpos' else abs(sweep)
num_turns = 6000

In [15]:
# Changing apertures of loss locations
line['bpcn.12508.a_aper'].shift_x = aperture_mvt
line['bpcn.12508.b_aper'].shift_x = aperture_mvt


line['bpcn.61108.a_aper'].shift_x = aperture_mvt
line['bpcn.61108.b_aper'].shift_x = aperture_mvt

In [ ]:
tidp = install_tidp(line)
tw = line.twiss()
# part = xp.generate_matched_gaussian_bunch(nemitt_x=2e-6,
#                                           nemitt_y=2e-6,
#                                           sigma_z=0.224, num_particles=num_particles, line=line)

#Particles
x_spacing = np.linspace(-0.025, 0.025, int(np.sqrt(num_particles)))
y_spacing = np.linspace(-0.025, 0.025, int(np.sqrt(num_particles)))

X, Y = np.meshgrid(x_spacing, y_spacing, indexing='xy')
x_norm = X.ravel()
y_norm = Y.ravel()

px_norm = np.zeros_like(x_norm)
py_norm = np.zeros_like(x_norm)
zeta = np.ones_like(x_norm)*tw.particle_on_co.zeta
delta = np.ones_like(x_norm)*tw.particle_on_co.delta

part = line.build_particles(x_norm=x_norm, px_norm=px_norm, y_norm=y_norm, py_norm=py_norm, nemitt_x=3.5e-6, nemitt_y=3.5e-6, zeta=zeta, delta=delta, at_element=i*100)

Slicing line:   0%|          | 0/33012 [00:00<?, ?it/s]

In [17]:
rf_sweep = xc.RFSweep(line)
rf_sweep.info(sweep=sweep, num_turns=num_turns)

line.discard_tracker()
line.build_tracker(_context=xo.ContextCpu(omp_num_threads='auto'))
line.scattering.enable()
rf_sweep.track(sweep=sweep, particles=part, num_turns=num_turns, time=True, with_progress=True, ele_start = i*100)
line.scattering.disable()

The current frequency is 200000000.0Hz, adding 1.0Hz per turn until 200006000.0 (for 6000 turns).
This sweep will move the center of the bucket with Δδ = -0.016896947679657257 (-2.81975385240468 buckets).
Compiling ContextCpu kernels...


ld: warning: duplicate -rpath '/Users/lisepauwels/miniforge3/envs/xcoll/lib' ignored
ld: warning: duplicate -rpath '/Users/lisepauwels/miniforge3/envs/xcoll/lib' ignored


Done compiling ContextCpu kernels.


Tracking:   0%|          | 0/6000 [00:00<?, ?it/s]

All particles lost at turn 2404, stopped sweep at 2380.0Hz.


In [31]:
tt = line.get_table()

In [33]:
tt.rows[tt.s==799.5901999999776]

Table: 0 rows, 11 cols

In [46]:
np.unique(part.at_element, return_counts=True)

(array([3772]), array([961]))

In [49]:
np.max(part.particle_id)

np.int64(960)

In [50]:
import json
# Get counts
elems, elem_counts = np.unique(part.at_element, return_counts=True)
turns, turn_counts = np.unique(part.at_turn, return_counts=True)

# Convert to JSON-serializable dict
data = {
    "at_element": {
        "values": elems.tolist(),
        "counts": elem_counts.tolist()
    },
    "at_turn": {
        "values": turns.tolist(),
        "counts": turn_counts.tolist()
    }
}

# Save to file
with open(f"counts_start_{0}.json", "w") as f:
    json.dump(data, f, indent=2)